In [3]:
import sys
sys.path.append('/MS3D')
from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.utils import common_utils
from pcdet.datasets import build_dataloader

def pretty_print(ap_dict):
    item_key = 'SOURCE\tTARGET\tMODEL\t'
    item_res = f'{cfg.DATA_CONFIG.DATASET.replace("Dataset","")}\t{cfg.DATA_CONFIG_TAR.DATASET.replace("Dataset","")}\t{cfg.MODEL.NAME}\t'
    for k,v in ap_dict.items():
        if ('VEHICLE' in k) or ('PEDESTRIAN' in k) or ('CYCLIST' in k):
            key = k[11:].replace('LEVEL_','L').replace('PEDESTRIAN','PED').replace('VEHICLE','VEH').replace('CYCLIST','CYC').replace(' ','')
            item_key += f'{key}\t'
            item_res += f'{ap_dict[k][0]*100:0.2f}\t'
    item_key += '\n'
    item_res += '\n'
    print(item_key)
    print(item_res)

cfg_file = '/MS3D/tools/cfgs/target_waymo/ms3d_nuscenes_voxel_rcnn_centerhead.yaml'

# Get target dataset
cfg_from_yaml_file(cfg_file, cfg)
logger = common_utils.create_logger('temp.txt', rank=cfg.LOCAL_RANK)
if cfg.get('DATA_CONFIG_TAR', False):
    dataset_cfg = cfg.DATA_CONFIG_TAR
    classes = cfg.DATA_CONFIG_TAR.CLASS_NAMES
    cfg.DATA_CONFIG_TAR.USE_PSEUDO_LABEL=False
    if dataset_cfg.get('USE_TTA', False):
        dataset_cfg.USE_TTA=False
else:
    dataset_cfg = cfg.DATA_CONFIG
    classes = cfg.CLASS_NAMES
dataset_cfg.DATA_SPLIT.test = 'train'
dataset_cfg.USE_CUSTOM_TRAIN_SCENES = True
dataset_cfg.SAMPLED_INTERVAL.test = 2

target_set, _, _ = build_dataloader(
            dataset_cfg=dataset_cfg,
            class_names=classes,
            batch_size=1, logger=logger, training=False, dist=False
        )


fatal: unsafe repository ('/MS3D' is owned by someone else)
To add an exception for this directory, call:

	git config --global --add safe.directory /MS3D
2023-05-22 06:36:05,109   INFO  Loading Waymo dataset


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2023-05-22 06:36:07,406   INFO  Total skipped info 0
2023-05-22 06:36:07,407   INFO  Total samples for Waymo dataset: 37680
2023-05-22 06:36:07,420   INFO  Total sampled samples for Waymo dataset: 18840


In [4]:
from pcdet.utils import box_fusion_utils
from tqdm import tqdm 
import numpy as np

dets_txt = '/MS3D/tools/cfgs/target_waymo/kbf_combinations/pvrcnn_c.txt'
print(f'classes: {cfg.DATA_CONFIG_TAR.CLASS_NAMES}')

ps_dict = {}
det_annos = box_fusion_utils.load_src_paths_txt(dets_txt)
print('Number of detectors: ', len(det_annos))
combined_dets = box_fusion_utils.combine_box_pkls(det_annos, score_th=0.1)

classes: ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']
Number of detectors:  12


aggregating_proposals: 100%|█████████████| 18840/18840 [01:13<00:00, 256.25it/s]


In [4]:
# from pcdet.utils import generate_ps_utils 
# generate_ps_utils.save_data(combined_dets, '/MS3D/tools/cfgs/target_waymo/kbf_combinations', 
#                             name=f"vox_c_combined_dets.pkl")

# Task:
1. Have diff discard/radius for diff classes
Maybe something like this?
```yaml
# Check that all lists are same length
classes: ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']
discard: [4,4,4, 4,4, 4]
radius: [2,2,2, 0.5,0.5, 0.3]
```
But if overlapping classes e.g. cyclist and pedestrian, you'd want the predominant class to be kept. Maybe it's okay to just match by class, then later on use NMS to keep the more confident classes?

2. Weighing by detection quantity

I wonder if there's some way to weigh by the number of detections it received? Low score but many detections make for a strong proposal.

#### Comments
Fine-tuning lyft -> waymo for ITSC submission was fine to use car,truck,bus -> vehicle,vehicle,vehicle. So I guess it doesn't affect the training to map it like this. I think it probably rendered the truck/bus heads useless since we labelled everything as 1 i.e. car. 

In [5]:
# Find frames with the most cyclists and pedestrians
num_cycs = {}
num_peds = {}
for enum, info in enumerate(target_set.infos):
    num_cyc = np.count_nonzero(info['annos']['name'] == 'Cyclist')
    num_ped = np.count_nonzero(info['annos']['name'] == 'Pedestrian')
    seq_name = '_'.join(info['frame_id'].split('_')[:-1])
    if seq_name not in num_cycs.keys():
        num_cycs[seq_name] = 0
    if seq_name not in num_peds.keys():
        num_peds[seq_name] = 0
    
    num_cycs[seq_name] += num_cyc
    num_peds[seq_name] += num_ped  
sorted_num_peds = {k: v for k, v in sorted(num_peds.items(), key=lambda item: item[1], reverse=True)}
sorted_num_cycs = {k: v for k, v in sorted(num_cycs.items(), key=lambda item: item[1], reverse=True)}    

In [6]:
ped_seqs = list(sorted_num_peds.keys())[:100]
cyc_seqs = list(sorted_num_cycs.keys())[:100]

In [7]:
ds_combined_dets = []
for frame in combined_dets:
    seq_name = '_'.join(frame['frame_id'].split('_')[:-1])
    if seq_name in cyc_seqs:
        ds_combined_dets.append(frame)
        
ds_combined_dets = ds_combined_dets[::5]        

### TODO: 
Have weights={}, weights['bw_c'], etc. where we specify an integer multiple of the src weights

In [8]:
from pcdet.utils.box_fusion_utils import *

def kde_fusion(boxes, src_weights, bw_c=1.0, bw_dim=2.0, bw_ry=0.1, bw_cls=0.5, bw_score=2.0):
    """
    Combines the centroids, dims, ry and scores of multiple predicted boxes
    Args:
        boxes: (N,9) np array. Boxes for filtering
        src_weights : (list). List of weights for each source detector

    Returns:
        combined box: (9) np array. A final box with params [x,y,z,dx,dy,dz,ry,score,class_id]

    Centroids are selected rather than aggregated as aggregating can lead to odd centering
    Rotations are selected as it is quite sensitive to aggregation
    Dimensions/score are the peak value of the KDE since we want to factor in the sizing/cls conf of diff detectors                    
    """
    def get_kde_value(data, src_weights, bw, return_max=False, verbose=False):
        """
        If data points are too similar, KDE will fail. This is because
        the covariance matrix is not invertible because the values are too close 
        (zero covriance on some diagonal elements).
        
        In these cases, we just take the weighted average.
        """
        if return_max:
            try:
                _, new_val = get_kde(data, return_max=return_max, weights=src_weights, bw_method=bw)
                return new_val
            except Exception as e:
                if verbose:
                    print(f'data:{data}, error:{e}')
                return np.average(data, axis=0, weights=src_weights)
        else:
            try:
                kde = get_kde(data, return_max=return_max, weights=src_weights, bw_method=bw)
                new_inds = get_sample_inds_with_max_likelihood(data, kde)    
                return new_inds
            except Exception as e:
                if verbose:
                    print(f'data:{data}, error:{e}')
                return None
        
    centroids = boxes[:,:3]
    det = np.linalg.det(np.cov(centroids.T, rowvar=1,bias=False))
    if det < 1e-7:
        new_cxy_inds = get_kde_value(centroids[:,:2], src_weights, bw_c, return_max=False)        
        if new_cxy_inds is not None:
            new_cxy = centroids[:,:2][new_cxy_inds]  
        else:
            new_cxy = np.average(centroids[:,:2], axis=0, weights=src_weights)
              
        new_cz_inds = get_kde_value(centroids[:,2], src_weights, bw_c, return_max=False)        
        if new_cz_inds is not None:
            new_cz = centroids[:,2][new_cz_inds]
        else:
            new_cz = np.average(centroids[:,2], axis=0, weights=src_weights)
        
        new_cxyz = np.hstack([new_cxy, new_cz])
    else:
        new_cxyz_inds = get_kde_value(centroids, src_weights, bw_c, return_max=False)
        if new_cxyz_inds is not None:
            new_cxyz = centroids[new_cxyz_inds]
        else:
            new_cxyz = np.average(centroids, axis=0, weights=src_weights)
    
    new_dx = get_kde_value(boxes[:,3], None, bw_dim, return_max=True)
    new_dy = get_kde_value(boxes[:,4], None, bw_dim, return_max=True)
    new_dz = get_kde_value(boxes[:,5], None, bw_dim, return_max=True)
        
    sin_rys = np.sin(boxes[:,6])
    ry_ind = get_kde_value(sin_rys, src_weights, bw_ry, return_max=False)    
    if ry_ind is not None:
        new_ry = boxes[:,6][ry_ind]
    else:
        new_ry = get_rotation_near_weighted_mean(boxes[:,6])
    

    cls_ind = get_kde_value(boxes[:,7], src_weights, bw_cls, return_max=False)
    if cls_ind is not None:
        new_class = boxes[:,7][cls_ind]
    else:
        # Return majority class
        unique, counts = np.unique(boxes[:,7], return_counts=True)
        new_class = int(unique[np.argmax(counts)])
    
    new_score = get_kde_value(boxes[:,8], src_weights, bw_score, return_max=True)
    
    combined_box = np.hstack([new_cxyz[0], new_cxyz[1], new_cxyz[2], new_dx, new_dy, new_dz, new_ry, new_class, new_score])
    return combined_box

In [139]:
for src_id, key in enumerate(det_annos.keys()):    
    print(f'{src_id:02d}: {key}')

00: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo1xyzt_custom190_notta
01: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo1xyzt_custom190_rwr
02: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo1xyzt_custom190_rwf
03: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo1xyzt_custom190_rwf_rwr
04: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo2xyzt_custom190_notta
05: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo2xyzt_custom190_rwr
06: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo2xyzt_custom190_rwf
07: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo2xyzt_custom190_rwf_rwr
08: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo4xyzt_custom190_notta
09: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo4xyzt_custom190_rwr
10: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo4xyzt_custom190_rwf
11: lyft_models.uda_voxel_rcnn_anchorhead.lyft10xyzt_waymo4xyzt_custom190_rwf_rwr
12: lyft_models.uda_voxel_rcnn_centerhead.lyft

In [9]:
def label_fusion(boxes_lidar, fusion_name, discard, radius, nms_thresh=0.05):
    """
    boxes_lidar (N,9): array of box proposals from src detectors
    
    return: 
        fused_boxes (N,9): fused box proposals
    """
    matched_inds_list = get_matching_boxes(boxes_lidar, discard=discard, radius=radius)
        
    # Aggregate these into one box
    combined_frame_boxes, num_matched_boxes = [], []
    for m_box_inds in matched_inds_list:

        matched_boxes = boxes_lidar[list(m_box_inds)]
        unique = np.unique(matched_boxes[:,:3].round(decimals=4), axis=0)
        if len(unique) < discard:
            continue

        src_weights = matched_boxes[:,8]                
        combined_box = kde_fusion(matched_boxes, src_weights=src_weights)        
        combined_frame_boxes.append(combined_box)
        num_matched_boxes.append(len(unique))
    
    if combined_frame_boxes:
        num_dets_per_box = np.array(num_matched_boxes)
        fused_boxes = np.array(combined_frame_boxes)
        nms_mask = nms(fused_boxes[:,:7], fused_boxes[:,8], thresh=nms_thresh)
        fused_boxes = fused_boxes[nms_mask]
        num_dets_per_box = num_dets_per_box[nms_mask]
    else:
        fused_boxes = np.empty((0,9))
        num_dets_per_box = np.empty(0)
        
    return fused_boxes.astype(np.float32), num_dets_per_box.astype(int) 

In [10]:
def get_ps_dict(ds_combined_dets, classes, cls_kbf_config, scale_conf_by_ndets, detector_weights=None):
    ps_dict = {}    
    for frame_boxes in tqdm(ds_combined_dets, total=len(ds_combined_dets)):        
            
        boxes_lidar = np.hstack([frame_boxes['boxes_lidar'],
                                 frame_boxes['class_ids'][...,np.newaxis],
                                 frame_boxes['score'][...,np.newaxis]])
        if detector_weights is not None:
            box_weights = detector_weights['heading'][frame_boxes['source_id']]
            boxes_lidar = np.hstack([boxes_lidar, box_weights[...,np.newaxis]])
        
        boxes_names = frame_boxes['names']
        unique_classes = np.unique(boxes_names)    
        ps_label_nms = []
        for cls in unique_classes:
            if cls not in classes:
                continue
            cls_mask = (boxes_names == cls)
            cls_boxes = boxes_lidar[cls_mask]
            score_mask = cls_boxes[:,8] > min_score
            
            cls_kbf_boxes, num_dets_per_box = label_fusion(cls_boxes, 'kde_fusion', 
                                           discard=cls_kbf_config[cls]['discard'], 
                                           radius=cls_kbf_config[cls]['radius'], 
                                           nms_thresh=cls_kbf_config[cls]['nms'], 
                                           weights=None)

            # E.g. car,truck,bus,motorcycle,bicycle,pedestrian (id: 1,2,3,4,5,6) -> Veh,Cyc,Ped (id: 1,4,6)
            cls_kbf_boxes[:,7] = cls_kbf_config[cls]['cls_id']

            # Scale confidence score by number of detections
            if scale_conf_by_ndets:
                coeff = conf_scaling_coeff(num_dets_per_box, num_boxes_at_unity)
                temp_frame_id = frame_boxes['frame_id']
                mask = cls_kbf_boxes[:,8] < cfg.SELF_TRAIN.SCORE_THRESH
                cls_kbf_boxes[mask,8] *= coeff[mask]
                cls_kbf_boxes[mask,8] = np.clip(cls_kbf_boxes[mask,8], 0, max_scale_score)

            ps_label_nms.extend(cls_kbf_boxes)

        if ps_label_nms:
            ps_label_nms = np.array(ps_label_nms)
        else:
            ps_label_nms = np.empty((0,9))

        # neg_th < score < pos_th: ignore for training but keep for update step
        pred_boxes = ps_label_nms[:,:7]
        pred_labels = ps_label_nms[:,7]
        pred_scores = ps_label_nms[:,8]
        ignore_mask = pred_scores < cfg.SELF_TRAIN.SCORE_THRESH
        pred_labels[ignore_mask] = -pred_labels[ignore_mask]
        gt_box = np.concatenate((pred_boxes,
                                pred_labels.reshape(-1, 1),
                                pred_scores.reshape(-1, 1)), axis=1)    

        gt_infos = {
            'gt_boxes': gt_box,
        }
        ps_dict[frame_boxes['frame_id']] = gt_infos
    
    return ps_dict

In [11]:
# ['Vehicle', 'Vehicle', 'Vehicle', 'Cyclist', 'Cyclist', 'Pedestrian']
classes=target_set.dataset_cfg.CLASS_NAMES
discard=[4]*6
radius=[1,1,1, 0.3,0.3, 0.2]
kbf_nms=[0.1,0.1,0.1,0.5,0.5,0.5,0.5]
detector_weights = {}
detector_weights['heading'] = np.array([1,1,1,1,2,2,2,2,2,2,2,2,1,1,1,1,3,3,3,3,3,3,3,3])
min_score = 0.3

# confidence scaling
scale_conf_by_ndets = True
num_boxes_at_unity = int(len(det_annos) * 0.75)
max_scale_score = cfg.SELF_TRAIN.SCORE_THRESH + 0.1

# Downsample
# ds_combined_dets = combined_dets[::3]

# Get class specific config
cls_kbf_config = {}
for enum, cls in enumerate(classes):
    if cls in cls_kbf_config.keys():
        continue
    cls_kbf_config[cls] = {}
    cls_kbf_config[cls]['cls_id'] = enum+1 # in OpenPCDet, cls_ids enumerate from 1
    cls_kbf_config[cls]['discard'] = discard[enum]
    cls_kbf_config[cls]['radius'] = radius[enum]
    cls_kbf_config[cls]['nms'] = kbf_nms[enum]

ps_dict = get_ps_dict(ds_combined_dets, ['Cyclist'], cls_kbf_config, 
                      scale_conf_by_ndets=scale_conf_by_ndets, detector_weights=detector_weights)

  0%|                                         | 1/1983 [00:00<00:02, 885.81it/s]


TypeError: label_fusion() got an unexpected keyword argument 'weights'

In [112]:
# from pcdet.utils import generate_ps_utils 
# generate_ps_utils.save_data(ps_dict, '/MS3D/tools/cfgs/target_waymo/kbf_combinations', 
#                             name=f"kbf_multiclass_exp1.pkl")

In [126]:
# Evaluate fused pred boxes
import copy
from tqdm import tqdm

dataset = target_set
pred_annos, gt_annos = [], []
eval_gt_annos = copy.deepcopy(dataset.infos)
for frame_id in tqdm(ps_dict.keys(), total=len(ps_dict.keys())):  
    boxes = ps_dict[frame_id]['gt_boxes'].copy()            
    boxes[:,:3] -= dataset.dataset_cfg.SHIFT_COOR # just for eval in this notebook
    p_anno = {"frame_id": frame_id,
              "name": np.array([dataset.dataset_cfg.CLASS_NAMES[int(abs(box[7]))-1] for box in boxes]),
              "pred_labels": np.array([abs(box[7]) for box in boxes]),
              "boxes_lidar": boxes[:,:7],
              "score": boxes[:,8]}
    pred_annos.append(p_anno)
    gt_annos.append(eval_gt_annos[dataset.frameid_to_idx[frame_id]]['annos'])

100%|██████████████████████████████████████| 695/695 [00:00<00:00, 55238.41it/s]


In [166]:
# ap_result_str, ap_dict = dataset.kitti_eval(pred_annos, eval_gt_annos, class_names=['Vehicle'])
# ap_result_str, ap_dict = dataset.evaluation(pred_annos, dataset.dataset_cfg.CLASS_NAMES, eval_metric='waymo')
from pcdet.datasets.waymo.waymo_eval import OpenPCDetWaymoDetectionMetricsEstimator
eval = OpenPCDetWaymoDetectionMetricsEstimator()

cls_names = dataset.dataset_cfg.CLASS_NAMES
cls_names = ['Vehicle','Pedestrian','Cyclist']
ap_dict = eval.waymo_evaluation(
    pred_annos, gt_annos, class_name=cls_names,
    distance_thresh=1000, fake_gt_infos=dataset.dataset_cfg.get('INFO_WITH_FAKELIDAR', False)
)

Start the waymo evaluation...
Number: (pd, 502597) VS. (gt, 483970)
Level 1: 403376, Level2: 80594


2023-05-22 00:01:34.665495: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2023-05-22 00:01:34.665525: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      
I0522 00:01:34.744335 18838 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:01:34.744354 18838 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:01:34.912678 18838 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:01:45.571396 18838 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:01:45.975847 18837 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:01:45.975864 18837 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:01:46.132743 18837 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:01:56.293432 18837 detection_metrics_o

I0522 00:05:28.777279 18840 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:05:29.224027 18844 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:05:29.224046 18844 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:05:29.323890 18844 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:05:39.361426 18844 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:05:39.759685 18839 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:05:39.759704 18839 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:05:39.848703 18839 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:05:49.853801 18839 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:05:50.255790 18844 detection_metrics_ops.cc:157] Computing detection metrics for 502

I0522 00:09:08.182754 18838 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:09:08.182770 18838 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:09:08.284351 18838 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:09:19.275123 18838 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:09:19.738257 18844 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:09:19.738272 18844 detection_metrics_ops.cc:159] Parsing prediction [502597,7][502597]
I0522 00:09:19.837486 18844 detection_metrics_ops.cc:168] Parsing ground truth [483970,7][483970]
I0522 00:09:30.405109 18844 detection_metrics_ops.cc:221] Done with computing detection metrics.
I0522 00:09:30.820116 18837 detection_metrics_ops.cc:157] Computing detection metrics for 502597 predicted boxes.
I0522 00:09:30.820133 18837 detection_metrics_ops.cc:159] Parsing prediction

In [167]:
pretty_print(ap_dict)

SOURCE	TARGET	MODEL	VEH_[0,30)_L1/AP	VEH_[0,30)_L1/APH	VEH_[0,30)_L2/AP	VEH_[0,30)_L2/APH	VEH_[30,50)_L1/AP	VEH_[30,50)_L1/APH	VEH_[30,50)_L2/AP	VEH_[30,50)_L2/APH	VEH_[50,+inf)_L1/AP	VEH_[50,+inf)_L1/APH	VEH_[50,+inf)_L2/AP	VEH_[50,+inf)_L2/APH	PED_[0,30)_L1/AP	PED_[0,30)_L1/APH	PED_[0,30)_L2/AP	PED_[0,30)_L2/APH	PED_[30,50)_L1/AP	PED_[30,50)_L1/APH	PED_[30,50)_L2/AP	PED_[30,50)_L2/APH	PED_[50,+inf)_L1/AP	PED_[50,+inf)_L1/APH	PED_[50,+inf)_L2/AP	PED_[50,+inf)_L2/APH	CYC_[0,30)_L1/AP	CYC_[0,30)_L1/APH	CYC_[0,30)_L2/AP	CYC_[0,30)_L2/APH	CYC_[30,50)_L1/AP	CYC_[30,50)_L1/APH	CYC_[30,50)_L2/AP	CYC_[30,50)_L2/APH	CYC_[50,+inf)_L1/AP	CYC_[50,+inf)_L1/APH	CYC_[50,+inf)_L2/AP	CYC_[50,+inf)_L2/APH	

NuScenes	Waymo	VoxelRCNN	74.66	74.20	72.80	72.35	48.30	47.68	42.21	41.66	25.11	24.52	18.12	17.69	34.26	19.34	31.89	17.99	21.69	12.18	18.37	10.31	11.37	6.15	7.80	4.22	45.72	41.31	44.89	40.56	28.48	23.54	25.21	20.84	14.01	12.27	11.94	10.46	



In [163]:
det_annos.keys()

dict_keys(['lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo1xyzt_notta', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo1xyzt_rwr', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo1xyzt_rwf', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo1xyzt_rwf_rwr', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo2xyzt_notta', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo2xyzt_rwr', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo2xyzt_rwf', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo2xyzt_rwf_rwr', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo4xyzt_notta', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo4xyzt_rwr', 'lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_wa

In [164]:
# Evaluate single pred annos
baseline = det_annos['lyft_models.uda_pv_rcnn_plusplus_resnet_centerhead.custom190_lyft10xyzt_waymo1xyzt_notta'][::3]
cls_names = ['Cyclist']
dataset = target_set
pred_annos, gt_annos = [], []
eval_gt_annos = copy.deepcopy(dataset.infos)
for p_anno in tqdm(baseline, total=len(baseline)):  
    p_anno['boxes_lidar'][:,:3] -= dataset.dataset_cfg.SHIFT_COOR # just for eval in this notebook
    pred_annos.append(p_anno)
    gt_annos.append(eval_gt_annos[dataset.frameid_to_idx[p_anno['frame_id']]]['annos'])    

100%|███████████████████████████████████| 6280/6280 [00:00<00:00, 154300.81it/s]


#### Scaling by number of detectors

This desmos grapher was useful for coming up with below https://www.desmos.com/calculator

At the end of the day we just wanted to map one range to another in a non-linear fashion where we can specify how many detections would give a scaling=1; or what the max scaling is.

In [75]:
def conf_scaling_coeff(num_boxes, num_boxes_at_unity, eq_type='sqrt', max_scaling=2.0):
    """
    This function returns the scaling coefficient to scale the confidence score based on
    the number of boxes proposed for an object. Two scaling equations are provided.
    
    Note: This might be bad if many detectors detect but wrongly detect -> adjust max_scaling according
    
    num_boxes (list of ints): number of input boxes for each kbf box fusion
    num_boxes_at_unity (int): The point at which the scaling coeff is equal to 1. 
                        Less than this and coeff < 1; More than this and coeff > 1.
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (1/np.sqrt(num_boxes_at_unity)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (1/num_boxes_at_unity) * num_boxes
        
    else:
        raise NotImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=1)

In [20]:
import matplotlib.pyplot as plt

# Set a config where you can enable/disable this
def min_boxes_conf_scaling(num_boxes, min_boxes, eq_type='sqrt', max_scaling=2.0):
    """
    For this function, at min_boxes, scaling is 1. We clip if it gets too large.    
    Adjustable param: min_boxes
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (1/np.sqrt(min_boxes)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (1/min_boxes) * num_boxes
        
    else:
        raise notImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=0)


def total_dets_conf_scaling(num_boxes, total_num_detectors, eq_type='sqrt', max_scaling=2.0):
    """
    For this function, at total_num_detectors, scaling is max_scaling. The point where scaling=1
    will change according to the total_num_detectors which may not be optimal.
    
    Adjustable param: total_num_detectors
    """
    if eq_type == 'sqrt':
        # Non-linear scaling with sqrt(x)        
        scaling = (max_scaling/np.sqrt(total_num_detectors)) * np.sqrt(num_boxes) 
        
    elif eq_type == 'linear':
        scaling = (max_scaling/total_num_detectors) * num_boxes
        
    else:
        raise notImplementedError
        
    return np.clip(scaling, a_max=max_scaling, a_min=0)


# min_dets = 20
# x = np.arange(0,30)
# # y = (1/np.sqrt(min_dets)) * np.sqrt(x) 
# y = 1/20 * x
# plt.plot(x,y)
# plt.grid(True)

# Non-linear scaling with sqrt(x)
# using 2/sqrt(num_det) means that at num_dets, scaling is max_scaling -> But we don't set min_dets
# Adjustable param: max_scaling
# num_detectors = 24
# max_scaling = 2
# x = np.arange(0,30)
# y = (max_scaling/np.sqrt(num_detectors)) * np.sqrt(x) 
# plt.plot(x,y)
# plt.grid(True)
scale1 = total_dets_conf_scaling(1, 6, eq_type='sqrt') # if total detectors is low, then the scaling is jumpy
scale2 = min_boxes_conf_scaling(6, 3, eq_type='sqrt') # works better for varying min_boxes
scale1, scale2

(0.8164965809277261, 1.4142135623730951)